In [1]:
import gc
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook
from glob import glob

from utils import loadpkl, to_json, scalingPredictions, getBestMultiple

%matplotlib inline

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [7]:
# search a best weight for 2 predictions
def getBestWeights(act, pred_lgbm, pred_xgb, output):
    search_range = np.arange(0.3, 0.6, 0.01)
    best_f1=0.0
    best_w = 0.5
    f1s = []
    for _w in search_range:
        # get predictions for each class
        cols_pred=[]
        _pred = pd.DataFrame()
        for i in range(0,12):
            _pred['pred_{}'.format(i)] = _w*pred_lgbm['pred_lgbm_plans{}'.format(i)]+ (1.0-_w)*pred_xgb['pred_xgb_plans{}'.format(i)]
            cols_pred.append('pred_{}'.format(i))
        
        # calc f1 score
        _f1 = f1_score(act, np.argmax(_pred[cols_pred].values,axis=1),average='weighted')
        f1s.append(_f1)
        print('w: {}, f1 score: {}'.format(_w,_f1))
        if _f1 > best_f1:
            best_f1 = _f1
            best_w = _w
        del _pred
            
    # plot thresholds
    plt.figure()
    plt.plot(search_range, f1s)
    plt.savefig(output)

    return best_w

In [3]:
# load predictions
pred_lgbm = loadpkl('../features/lgbm_pred.pkl')
pred_xgb = loadpkl('../features/xgb_pred.pkl')
plans = loadpkl('../features/plans.pkl')

# define columns name list
cols_pred_lgbm = ['pred_lgbm_plans{}'.format(i) for i in range(0,12)]
cols_pred_xgb = ['pred_xgb_plans{}'.format(i) for i in range(0,12)]
cols_transport_mode = ['plan_{}_transport_mode'.format(i) for i in range(0,7)]

# blend predictions
pred = pred_lgbm[['sid','click_mode']]
pred = pd.merge(pred, plans[cols_transport_mode+['sid','plan_num_plans']],on='sid', how='left')

del plans
gc.collect()

# scaling
pred_lgbm[cols_pred_lgbm] = scalingPredictions(pred_lgbm[cols_pred_lgbm])
pred_xgb[cols_pred_xgb] = scalingPredictions(pred_xgb[cols_pred_xgb])

# reset index
pred_lgbm.reset_index(inplace=True,drop=True)
pred_xgb.reset_index(inplace=True,drop=True)

for i in range(1,12):
    tmp = np.zeros(len(pred))
    for c in cols_transport_mode:
        tmp += (pred[c]==i).astype(int)
    pred_lgbm['pred_lgbm_plans{}'.format(i)]=pred_lgbm['pred_lgbm_plans{}'.format(i)]*(tmp>0)
    pred_xgb['pred_xgb_plans{}'.format(i)]=pred_xgb['pred_xgb_plans{}'.format(i)]*(tmp>0)

/media/fujiwara/2THDD/KDD-Cup-2019/src/utils.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['pred_min'] = pred_df[cols_pred].min(axis=1)


In [4]:
oof_pred_lgbm = pred_lgbm[pred_lgbm.click_mode.notnull()]
oof_pred_xgb = pred_xgb[pred_xgb.click_mode.notnull()]

In [8]:
w = getBestWeights(oof_pred_lgbm.click_mode, oof_pred_lgbm, oof_pred_xgb, '../imp/weight.png')

cols_pred =[]
for i in range(0,12):
    pred['pred_{}'.format(i)] = w*pred_lgbm['pred_lgbm_plans{}'.format(i)]+ (1.0-w)*pred_xgb['pred_xgb_plans{}'.format(i)]
    cols_pred.append('pred_{}'.format(i))

w: 0.3, f1 score: 0.6676654072143018
w: 0.31, f1 score: 0.6676974459145322
w: 0.32, f1 score: 0.6676968920211782
w: 0.33, f1 score: 0.6677062948529124
w: 0.34, f1 score: 0.6677142840359317
w: 0.35000000000000003, f1 score: 0.6677149717766634
w: 0.36000000000000004, f1 score: 0.6677012538733206
w: 0.37000000000000005, f1 score: 0.6677061406169327
w: 0.38000000000000006, f1 score: 0.6677086675539763
w: 0.39000000000000007, f1 score: 0.6677024731767333
w: 0.4000000000000001, f1 score: 0.66772409316681
w: 0.4100000000000001, f1 score: 0.6677295167677656
w: 0.4200000000000001, f1 score: 0.667740504689243
w: 0.4300000000000001, f1 score: 0.6677364556680616
w: 0.4400000000000001, f1 score: 0.6677196862357659
w: 0.4500000000000001, f1 score: 0.6676995025329242
w: 0.46000000000000013, f1 score: 0.6676827280190475
w: 0.47000000000000014, f1 score: 0.6676831797503814
w: 0.48000000000000015, f1 score: 0.6676931471042151
w: 0.49000000000000016, f1 score: 0.6676925023230756
w: 0.5000000000000002, f1

In [7]:
pred['recommend_mode'] = np.argmax(pred[cols_pred].values,axis=1)
pred['recommend_mode'][(pred['plan_num_plans']==1)&(pred['recommend_mode']!=0)] = pred['plan_0_transport_mode'][(pred['plan_num_plans']==1)&(pred['recommend_mode']!=0)]

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
# split train & test
oof_pred = pred[pred['click_mode'].notnull()]
sub_pred = pred[pred['click_mode'].isnull()]

oof_pred['pred_0']*=1.57
#oof_pred['pred_4']*=1.13
oof_pred['recommend_mode'] = np.argmax(oof_pred[cols_pred].values,axis=1)

# out of fold score
oof_f1_score = f1_score(oof_pred['click_mode'], oof_pred['recommend_mode'],average='weighted')
oof_f1_score

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0.6725579034495828

In [9]:
w_click = oof_pred.click_mode.astype(int).value_counts(normalize=True).to_dict()
w_recommend = oof_pred.recommend_mode.value_counts(normalize=True).to_dict()

In [10]:
for i, c in enumerate(cols_pred):
    print(w_click[i]/w_recommend[i])

14.158408408408407
0.834002962962963
0.9003779857909007
3.817984496124031
27.344902386117138
0.8549717290308639
2.024057328101007
0.8355305328832101
1.3160878809355068
0.8026676741626558
0.7447330230696091
0.7388666423977672


In [17]:
def getBestMultiple(pred_df, col, cols_pred, output):
    best_f1=0.0
    best_m = 1.0
    f1s = []
    for _m in np.arange(1.0,2.0,0.01):
        tmp_pred = pred_df[cols_pred]
        tmp_pred[col] *= _m
        _f1 = f1_score(pred_df['click_mode'], np.argmax(tmp_pred.values,axis=1),average='weighted')
        f1s.append(_f1)
        print('multiple: {}, f1 score: {}'.format(_m,_f1))
        if _f1 > best_f1:
            best_f1 = _f1
            best_m = _m
        del tmp_pred
    # plot thresholds
    plt.figure()
    plt.plot(np.arange(1.0,2.0,0.01), f1s)
    plt.savefig(output)

    return best_m

In [18]:
getBestMultiple(oof_pred,'pred_0',cols_pred,'../imp/multiple0.png')

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


multiple: 1.0, f1 score: 0.6673437324314919
multiple: 1.01, f1 score: 0.6674620386482394
multiple: 1.02, f1 score: 0.6676478242703983
multiple: 1.03, f1 score: 0.667745331680426
multiple: 1.04, f1 score: 0.6678666300910207
multiple: 1.05, f1 score: 0.6680348738132594
multiple: 1.06, f1 score: 0.6682212256767697
multiple: 1.07, f1 score: 0.6683184877800256
multiple: 1.08, f1 score: 0.6684017766931544
multiple: 1.09, f1 score: 0.6685218299709454
multiple: 1.1, f1 score: 0.6685901058321567
multiple: 1.11, f1 score: 0.668736386652501
multiple: 1.12, f1 score: 0.6688769707393154
multiple: 1.1300000000000001, f1 score: 0.6690142783418992
multiple: 1.1400000000000001, f1 score: 0.669099832732028
multiple: 1.1500000000000001, f1 score: 0.6692511006718468
multiple: 1.1600000000000001, f1 score: 0.6693580138297079
multiple: 1.1700000000000002, f1 score: 0.6694660268610858
multiple: 1.1800000000000002, f1 score: 0.6695931355909637
multiple: 1.1900000000000002, f1 score: 0.6696740653990918
multipl

1.5700000000000005

In [11]:
oof_pred[cols_pred]

pred_0    pred_1    pred_2    pred_3    pred_4    pred_5    pred_6  \
0       0.122563  0.089278  0.159251  0.091397  0.066360  0.000000  0.000000   
1       0.086975  0.000000  0.597601  0.073392  0.064322  0.066188  0.074504   
2       0.128660  0.128148  0.000000  0.106861  0.089754  0.000000  0.000000   
3       0.109485  0.000000  0.548793  0.063473  0.066186  0.000000  0.066140   
4       0.107338  0.134104  0.000000  0.092354  0.088496  0.000000  0.000000   
5       0.198230  0.000000  0.000000  0.622710  0.000000  0.000000  0.000000   
6       0.098168  0.000000  0.579484  0.074275  0.075442  0.064276  0.070997   
7       0.111831  0.103498  0.485162  0.103361  0.075220  0.000000  0.000000   
8       0.089926  0.085665  0.519522  0.072608  0.070499  0.000000  0.051159   
9       0.113479  0.462964  0.000000  0.078911  0.067064  0.122583  0.124648   
10      0.081760  0.000000  0.544426  0.067078  0.071573  0.000000  0.000000   
11      0.080514  0.098365  0.543889  0.062868  0.053288  0.000000  0.058280   
12      0.125628  0.205946  0.000000  0.000000  0.000000  0.000000  0.000000   
13      0.121637  0.092179  0.146459  0.067064  0.060369  0.370551  0.120049   
14      0.140326  0.424061  0.000000  0.114537  0.075255  0.091211  0.125635   
15      0.131187  0.110761  0.000000  0.111623  0.090583  0.000000  0.000000   
16      0.124871  0.000000  0.557645  0.104651  0.076210  0.000000  0.000000   
17      0.143763  0.000000  0.000000  0.183378  0.097861  0.121234  0.133405   
18      0.155795  0.000000  0.135493  0.100961  0.114397  0.000000  0.070920   
19      0.113566  0.132884  0.431057  0.124000  0.074034  0.000000  0.000000   
20      0.126114  0.268918  0.000000  0.000000  0.000000  0.000000  0.000000   
21      0.125308  0.108521  0.417949  0.065610  0.070054  0.100394  0.094749   
22      0.132212  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
23      0.098786  0.000000  0.000000  0.079403  0.069007  0.000000  0.000000   
24      0.093427  0.000000  0.607651  0.076299  0.058351  0.000000  0.000000   
25      0.221385  0.000000  0.000000  0.117933  0.000000  0.425667  0.167115   
26      0.113214  0.126432  0.000000  0.076058  0.000000  0.513656  0.114144   
27      0.101552  0.104695  0.137680  0.057393  0.067041  0.410581  0.098185   
28      0.109970  0.000000  0.481961  0.088477  0.062331  0.000000  0.000000   
29      0.089648  0.000000  0.565348  0.078628  0.067393  0.000000  0.000000   
...          ...       ...       ...       ...       ...       ...       ...   
491024  0.094928  0.148442  0.116591  0.071890  0.065417  0.000000  0.086188   
491025  0.110867  0.000000  0.529903  0.088959  0.080246  0.000000  0.000000   
491026  0.082272  0.097376  0.544656  0.065386  0.073866  0.000000  0.000000   
491027  0.094485  0.105510  0.000000  0.072951  0.070055  0.000000  0.000000   
491028  0.082815  0.112843  0.000000  0.080503  0.068027  0.000000  0.000000   
491029  0.123696  0.000000  0.635347  0.092879  0.085842  0.000000  0.000000   
491030  0.095089  0.122650  0.000000  0.078431  0.086833  0.000000  0.073627   
491031  0.082748  0.000000  0.000000  0.071315  0.061873  0.000000  0.000000   
491032  0.090273  0.084012  0.583693  0.065139  0.063164  0.000000  0.000000   
491033  0.126747  0.000000  0.000000  0.094019  0.000000  0.612387  0.092652   
491034  0.129374  0.539269  0.000000  0.085964  0.091384  0.000000  0.115670   
491035  0.117918  0.150556  0.000000  0.085556  0.069151  0.276481  0.273233   
491036  0.102018  0.000000  0.000000  0.076940  0.075161  0.000000  0.000000   
491037  0.149977  0.442682  0.000000  0.149647  0.121311  0.000000  0.102557   
491038  0.098576  0.124005  0.000000  0.081191  0.084769  0.000000  0.000000   
491039  0.170837  0.000000  0.000000  0.155207  0.099550  0.000000  0.000000   
491040  0.090281  0.487166  0.000000  0.079239  0.090721  0.000000  0.000000   
491041  0.101001  0.000000  0.565214  0.090322  0.080702  0.000000  0.000000   
491042  0.